In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb


We will be looking at school specific data from across the United States. Our goal is to see if any schools resemble eachother. Often schools are described as "good", but if you are not familiar with the area you have little idea what that definition of "good" means. We want to explore the school graduation rates, diversity, expulsion rates, and teacher ratios to get an idea of what a "good" school is.

If we are also able to cluster like schools together, you may be able to see how your current school (something you are familiar with) compares to other schools you are looking to move into.

## Data sources:

2015-2016 graduation rates: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr

2015-2016 school location, characteristics, staff, and lunch programs: https://nces.ed.gov/ccd/pubschuniv.asp

2015-2016 school achievement: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html#acgr

2015-2016 suspenion rates: https://ocrdata.ed.gov/DataAnalysisTools/DataSetBuilder?Report=4
for this analysis, we only focused on the greenville district schools, but you could select other areas of interest.
    

In [23]:
#load data
path='/home/natalie/Documents/SchoolAPI/Data_Sources/'
grad_2015=pd.read_csv(path+'graduation_rates/2015_school_graduation.csv', parse_dates=True)


In [24]:
#school specific data
school_lunch=pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_lunches_ccd_sch_033_1516.csv',  parse_dates=True)
#school_char=pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_characteristics_ccd_sch_129_1516_w_2a_011717.csv', parse_dates=True)
school_dir =pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_directory_ccd_sch_029_1516_w_2a_011717.csv',  parse_dates=True)
school_member=pd.read_csv(path+ 'CCD_comoncoredata_nonfiscal/schools_membership_ccd_sch_052_1516_w_2a_011717.csv' ,  parse_dates=True)
school_staff = pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_staff_ccd_sch_059_1516_w_2a_011717.csv')

/home/natalie/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,14,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/natalie/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
school_staff = pd.read_csv(path+'CCD_comoncoredata_nonfiscal/school_staff_ccd_sch_059_1516_w_2a_011717.csv')

In [26]:
#School achievement results for math and reading

math=pd.read_csv(path+'Subject_assesments/school_math-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)
reading= pd.read_csv(path+'Subject_assesments/rla-achievement-sch-sy2015-16.csv', parse_dates=True, low_memory=False)

# percent is the % above the state's "proficient" level
#colums starting with F or M denote breakdowns by gender


In [27]:
#greenville suspensions
suspensions=pd.read_csv(path+'civil_rights/suspensions_greenville_2015.csv')

In [28]:
suspensions.head()
#n = individual counts of students
#otherwise in percentage
#times white students is a multiplier of how much more likely they are to be suspended than a white student

,District ID,District,State,School_ID,School Name,AME_n_susp,ASI_n_susp,HI PAC_n_susp,BLA_n_susp,HIS_n_susp,...,HIS_OSS_times_white,TWO_OSS_times_white,AME_n_enrollment,ASI_n_enrollment,HI_PAC_n_enrollment,BLA_n_enrollment,HIS_n_enrollment,WHI_n_enrollment,TWO_n_enrollment,TOT_n_enrollment
0,4502310,Greenville 01,SC,450231000049,Heritage Elementary,0,0,0,2,0,...,0.00,0.00,0,4,0,61,16,565,25,671
1,4502310,Greenville 01,SC,450231000114,Oakview Elementary,0,0,0,6,0,...,0.00,0.00,10,79,0,91,37,697,46,960
2,4502310,Greenville 01,SC,450231000449,Chandler Creek Elementary,0,2,0,12,6,...,1.84,0.00,19,4,0,214,190,526,64,1017
3,4502310,Greenville 01,SC,450231000455,Mauldin Middle,0,2,0,37,4,...,0.77,3.62,4,37,2,214,82,631,55,1025
4,4502310,Greenville 01,SC,450231000459,Riverside Middle,0,0,0,20,7,...,1.36,3.53,2,76,2,121,109,805,49,1164


## Lets pair down the data to only include the variables we are interested in, and then do the cleaning.

### For the math and reading dataste,these variables are:

STNAM = state name

LEANM = education agency name

NCESSCH = unique school ID

SCHNAM = school name

ALL_MTH00NUMVALID_1516  = number of students aggregated across all grades who took test
ALL_MTH00PCTPROF_1516= percentage of all students who are proficient in math across all grades

F_MTH00NUMVALID_1516 = females who took math exam across all grades
F_MTH00PCTPROF_1516 = females proficient in math across all grades

M_MTH00NUMVALID_1516 = males who took math exam across all grades
M_MTH00PCTPROF_1516 = males proficient in math across all grades

MBL_MTHHSNUMVALID_1516 = number of black students in high school who took math examns
MBL_MTHHSPCTPROF_1516 = percent of black students in high school who are proficient in math

MWH_MTHHSNUMVALID_1516 = Number of white students in high school who took math exams
MWH_MTHHSPCTPROF_1516 = percent of white students in high school who are proficient in math

same thing for reading, but MTH is replaced with RLA

In [29]:
#keep only variables listed above

math_keep=['STNAM', 'LEANM', 'NCESSCH', 'SCHNAM', 'ALL_MTH00NUMVALID_1516', 'ALL_MTH00PCTPROF_1516', 'F_MTH00NUMVALID_1516', 'F_MTH00PCTPROF_1516', 'M_MTH00NUMVALID_1516', 'M_MTH00PCTPROF_1516', 'MBL_MTHHSNUMVALID_1516', 'MBL_MTHHSPCTPROF_1516' , 'MWH_MTHHSNUMVALID_1516', 'MWH_MTHHSPCTPROF_1516']
math=math[math_keep]


reading_keep=['STNAM', 'LEANM', 'NCESSCH', 'SCHNAM', 'ALL_RLA00NUMVALID_1516', 'ALL_RLA00PCTPROF_1516', 'F_RLA00NUMVALID_1516', 'F_RLA00PCTPROF_1516', 'M_RLA00NUMVALID_1516', 'M_RLA00PCTPROF_1516', 'MBL_RLAHSNUMVALID_1516', 'MBL_RLAHSPCTPROF_1516' , 'MWH_RLAHSNUMVALID_1516', 'MWH_RLAHSPCTPROF_1516']
reading=reading[reading_keep]

math.head(11)

,STNAM,LEANM,NCESSCH,SCHNAM,ALL_MTH00NUMVALID_1516,ALL_MTH00PCTPROF_1516,F_MTH00NUMVALID_1516,F_MTH00PCTPROF_1516,M_MTH00NUMVALID_1516,M_MTH00PCTPROF_1516,MBL_MTHHSNUMVALID_1516,MBL_MTHHSPCTPROF_1516,MWH_MTHHSNUMVALID_1516,MWH_MTHHSPCTPROF_1516
0,ALABAMA,Albertville City,10000500870,Ala Avenue Middle Sch,707,36,347,39,360,34,NaN,NaN,NaN,NaN
1,ALABAMA,Albertville City,10000500871,Albertville High Sch,307,15,161,15-19,146,10-14,9,LT50,177,20-24
2,ALABAMA,Albertville City,10000500879,Evans Elem Sch,772,43,389,46,383,40,NaN,NaN,NaN,NaN
3,ALABAMA,Albertville City,10000500889,Albertville Elem Sch,852,55,421,54,431,57,NaN,NaN,NaN,NaN
4,ALABAMA,Marshall County,10000600193,Kate Duncan Smith DAR Middle,443,36,206,30-34,237,35-39,NaN,NaN,NaN,NaN
5,ALABAMA,Marshall County,10000600872,Asbury Sch,299,33,141,35-39,158,30-34,NaN,NaN,46,LE10
6,ALABAMA,Marshall County,10000600876,Claysville Jr High Sch,37,40-49,20,40-59,17,21-39,NaN,NaN,NaN,NaN
7,ALABAMA,Marshall County,10000600877,Douglas Elem Sch,494,55,235,60-64,258,45-49,NaN,NaN,NaN,NaN
8,ALABAMA,Marshall County,10000600878,Douglas High Sch,148,6-9,66,6-9,82,6-9,3,PS,108,6-9
9,ALABAMA,Marshall County,10000600880,Grassy Elem Sch,235,40,102,40-44,133,40-44,NaN,NaN,NaN,NaN


### School directory dataset, we are interested in these variables:
    
FIPST = state name
NCESSCH = school ID
LCITY = location city
LZIP = location zip code
WEBSITE = website
G1OFFERED = grade 1 offered
G8OFFERED = grade 8 offered
G12FFERED = grade 12 offered
NOGRADES = no grades offered, if true this school is not operational


In [30]:
dir_keep=['FIPST', 'NCESSCH', 'LCITY', 'LZIP', 'WEBSITE', 'G1OFFERED', 'G8OFFERED', 'G12OFFERED' ,'NOGRADES']
school_dir=school_dir[dir_keep]

### School membership

This is breakdown in grade by race and gender

NCESSCH = school ID
SCH_NAME = school name
GO1 = number of students in grade 1
GO8 = number of students in grade 8
G12 = number of students in grade 12
TOTAL = total students in all grades in school
AM = number of all students in school american indian
AMALM = '' male
AMALF = "" female
AS = number of all students asian
ASALM = "" male
ASALF = "" female
HI = number of all students Hispanic
HIALM
HIALF
BL = number of all students black
BLALM
BLALF
WH = number of all stduents white
WHALM
WHALF
HP = number of all students hawaiian
HPALM
HPALF
TR = number of all students 2 or more races
TRALM
TRALF

AS01M = american indian Male
AS01F = american indian female
HI01M = hispanic Male
HI01F = hispanic female grade1
BL01M = black male grade 1
BL01F = black Female grade1
WH01M = white male grade 1
WH01F = white female grade 1
HP01M = hawaiian/pacific islander male
HP01F = hawaiian/pacific islander Female
TR01M = two or more races male grade 1
TR01F = two or more races female grade 1



In [31]:
school_mem_keep=['NCESSCH', 'SCH_NAME', 'G01', 'G08', 'G12', 'TOTAL', 'AM', 'AMALM', 'AMALF', 'AS', 'ASALM', 'ASALF', 'HI', 'HIALM', 'HIALF', 'BL', 'BLALM', 'BLALF', 'WH', 'WHALM', 'WHALF', 'HP', 'HPALM', 'HPALF', 'TR', 'TRALM', 'TRALF']
school_member=school_member[school_mem_keep]

### School staff dataset, we are interested in these variables:

NCESSCH = school ID
FTE = full time equivalent teachers (value -1 is missing)

In [32]:
school_staff=school_staff[['NCESSCH','FTE']]

### Graduation rate dataset

NCESSCH = school ID

MAM = American Indian/Alaska Native students

MAS = Asian/Pacific Islander

MHI = Hispanic/Latino

MBL = Black

MWH = White

MTR = Two or more races

CWD = disabilities

ECD = Economically disadvanted

LEP = Limited English Proficiency

COHORT = number of students

RATE = percentage in that group out of the entire student population of that school

In [33]:
grad_keep=['NCESSCH', 'ALL_COHORT_1516', 'ALL_RATE_1516', 'MAM_RATE_1516', 'MAS_RATE_1516', 'MBL_RATE_1516', 'MHI_RATE_1516', 'MTR_RATE_1516', 'MWH_RATE_1516', 'CWD_RATE_1516', 'ECD_RATE_1516', 'LEP_RATE_1516']
grad_2015=grad_2015[grad_keep]

### Suspensions dataset

School_ID = school ID
TOT_n_susp = total number of suspensions in that school
TOT_OSS_rate = total number of suspensions in school/ school enrollment

" " OSS_Rate = number of students in this race suspended/ total student of that race enrollment in school

" " OSS_times_white = the rate of suspensions for this race is xx times that of white students

In [34]:
susp_keep=['School_ID', 'TOT_n_susp', 'AME_OSS_times_white', 'ASI_OSS_times_white', 'HI_PAC_OSS_times_white', 'BLA_OSS_times_white', 'HIS_OSS_times_white', 'TWO_OSS_times_white', 'TOT_n_enrollment']

suspensions=suspensions[susp_keep]

In [35]:
suspensions.head()

,School_ID,TOT_n_susp,AME_OSS_times_white,ASI_OSS_times_white,HI_PAC_OSS_times_white,BLA_OSS_times_white,HIS_OSS_times_white,TWO_OSS_times_white,TOT_n_enrollment
0,450231000049,25,0.0,0.00,0.0,0.79,0.00,0.00,671
1,450231000114,16,0.0,0.00,0.0,4.52,0.00,0.00,960
2,450231000449,29,0.0,28.41,0.0,3.24,1.84,0.00,1017
3,450231000455,94,0.0,0.83,0.0,2.71,0.77,3.62,1025
4,450231000459,72,0.0,0.00,0.0,3.55,1.36,3.53,1164


# Data Cleaning

#### 1. the rates of graduation or math/reading competence can be given in values '50-60', or 'GE50' = greater than 50%, or 'PS' which means the data was intentially hidden so you could not focus in on a very small group (e.g., 2 students are hispanic, so any data on these 2 students would wrongly expose their identity). To be conservative, we will select the lower range 50-60, we would select '50' to represent the graduation rate. For values with GE greater than or LE less than, we simply select the value stated, so GE80 becomes '80'.

In [36]:
import re
grad_variables=list(grad_2015)


In [37]:
grad_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 12 columns):
NCESSCH            23090 non-null int64
ALL_COHORT_1516    23090 non-null int64
ALL_RATE_1516      23090 non-null object
MAM_RATE_1516      19434 non-null object
MAS_RATE_1516      20357 non-null object
MBL_RATE_1516      21736 non-null object
MHI_RATE_1516      22034 non-null object
MTR_RATE_1516      20705 non-null object
MWH_RATE_1516      22716 non-null object
CWD_RATE_1516      22656 non-null object
ECD_RATE_1516      22936 non-null object
LEP_RATE_1516      20740 non-null object
dtypes: int64(2), object(10)
memory usage: 2.1+ MB


In [39]:
replace_list=['GE', 'LE', 'LT']

for word in replace_list:
    reading=reading.replace(word, '', regex=True)
    math=math.replace(word, '', regex=True)
    grad_2015=grad_2015.replace(word, '', regex=True) #regex=TRUE checks for partial match, otherwise it would need to match it 100%


In [40]:
grad_2015.head()

,NCESSCH,ALL_COHORT_1516,ALL_RATE_1516,MAM_RATE_1516,MAS_RATE_1516,MBL_RATE_1516,MHI_RATE_1516,MTR_RATE_1516,MWH_RATE_1516,CWD_RATE_1516,ECD_RATE_1516,LEP_RATE_1516
0,10000500871,296,92,PS,PS,50,95,.,90-94,60-79,80-84,50
1,10000600872,67,95,PS,.,PS,80,.,90,50,90,PS
2,10000600878,153,85-89,.,.,PS,90,.,85-89,50,85-89,PS
3,10000600883,120,80-84,.,.,.,PS,.,80-84,21-39,60-69,.
4,10000601585,94,85-89,PS,PS,PS,PS,PS,85-89,50,80-89,.


In [42]:
na_list=['.', 'PS']

for word in na_list:
    reading=reading.replace(word, np.nan)
    math=math.replace(word, np.nan)
    grad_2015=grad_2015.replace(word, np.nan) #here we want an exact match, so we do not include reges

In [45]:
#removehe the ranges, instead select the lowest value in the range, if rate is 80-85, we only want to keep 80
#grad_2015['ALL_RATE_1516'] = grad_2015['ALL_RATE_1516'].str.split('-').str[0]

r=re.compile(".*RATE")
rate_variables = list(filter(r.match, grad_variables)) # Read Note

for variable in rate_variables:
    grad_2015[variable]= grad_2015[variable].str.split('-').str[0]

r=re.compile(".*PROF")
prof_variables=list(filter(r.match, list(math)))

for variable in prof_variables:
    math[variable]= math[variable].str.split('-').str[0]

In [46]:
#same thing remove dashes in reading variables
r=re.compile(".PROF")
prof_variables=list(filter(r.match, list(reading)))

for variable in prof_variables:
    reading[variable]= reading[variable].str.split('-').str[0]


2. Create class variables in school directory dataset

If a school offers grade 12 and not grade 1 it is considered a High School. If a school offers grade 1 but not grade 8 it is considered elementary. If a school offers grade 8 but not 12 or 1, it is a middle school

In [31]:
#G12OFFERED = YES then this is a high school
#G1OFFERED = YES and G12OFFERED=NO then this is an elementary school
#first replace yes and no with 1 and 0, respectively

school_dir=school_dir.replace(['yes', 'Yes'], 1, regex=True)
#school_dir1=school_dir.replace('Yes', 1)
school_dir=school_dir.replace('No', 0, regex=True)

In [32]:
school_dir['type_school']=np.nan

#df['elderly'] = np.where(df['age']>=50, 'yes', 'no')
      
school_dir['type_school'] = np.where((school_dir['G1OFFERED']==1) & (school_dir['G12OFFERED']==0) , 'elementary',
                np.where((school_dir['G1OFFERED']==0) & (school_dir['G12OFFERED']==1) , 'high_school', 
                    np.where((school_dir['G1OFFERED']==0) & (school_dir['G12OFFERED']==0)& (school_dir['G8OFFERED']==1) , 'middle',  np.nan)))


In [33]:
school_dir[['G1OFFERED', 'G8OFFERED', 'G12OFFERED',  'type_school']].head(10)

,G1OFFERED,G8OFFERED,G12OFFERED,type_school
0,0,1,1,high_school
1,0,0,0,nan
2,0,1,1,high_school
3,0,1,1,high_school
4,0,1,1,high_school
5,0,1,1,high_school
6,0,0,0,nan
7,0,0,0,nan
8,0,1,0,middle
9,0,0,0,nan


In [ ]:
grad_2015.info()

In [13]:
#Change all the Cohort variables (those with specific numbers of students to float instead of object)
r=re.compile(".*COHORT")
cohort_variables = list(filter(r.match, grad_variables))

for variable in rate_variables:
    grad_2015[variable]=grad_2015[variable].astype('float')
    
for variable in cohort_variables:
    grad_2015[variable]=grad_2015[variable].astype('float')

## The "All_Cohort" variable is all students in that school, then they are broken down into racial groups.

MAM = American Indian/Alaska Native students

MAS = Asian/Pacific Islander

MHI = Hispanic/Latino

MBL = Black

MWH = White

MTR = Two or more races

CWD = disabilities

ECD = Economically disadvanted

LEP = Limited English Proficiency

In [1]:
grad_2015.describe() #everything we would expect is numeric and can be summarized. These columns can also be used to creat a distance matrix between the different schools.

NameError: name 'grad_2015' is not defined

Clustering algorithims cannot handle missing data, what data is missing?

In [59]:
na_list=grad_2015.isna().sum()
na_list/len(grad_2015) # percentage
#Because many of these race groups are so small the PS identifier was used, and then converted to NA.

STNAM              0.0
FIPST              0.0
LEAID              0.0
LEANM              0.0
NCESSCH            0.0
SCHNAM             0.0
ALL_COHORT_1516    0.0
ALL_RATE_1516      0.0
MAM_COHORT_1516    0.0
MAM_RATE_1516      0.0
MAS_COHORT_1516    0.0
MAS_RATE_1516      0.0
MBL_COHORT_1516    0.0
MBL_RATE_1516      0.0
MHI_COHORT_1516    0.0
MHI_RATE_1516      0.0
MTR_COHORT_1516    0.0
MTR_RATE_1516      0.0
MWH_COHORT_1516    0.0
MWH_RATE_1516      0.0
CWD_COHORT_1516    0.0
CWD_RATE_1516      0.0
ECD_COHORT_1516    0.0
ECD_RATE_1516      0.0
LEP_COHORT_1516    0.0
LEP_RATE_1516      0.0
DATE_CUR           0.0
kmeans             0.0
dtype: float64

In [17]:
# Fill missing values with mean column values in those columns
grad_2015.fillna(grad_2015.mean(), inplace=True)

### Clustering on just the graduation data did not seem to help, let's see if pulling in other data will improve our model.



Merge all the data for individual schools together, using 'NCESSCH' as the key id variable.

In [ ]:
all_data=pd.merge(df_new, grad_2015, on='subject_id')